In [1]:
# Read cnn model in different structure
import torch 
mnist_models = []
cifar10_models = []

def load_spec_model(module, model_index):
    model = getattr(module, 'Model'+model_index)
    return model

for i in range(6):
    # params = torch.load('/home/ubuntu/date/hdd4/shadow_model_ckpt/mnist/models%d/shadow_benign_0.model'%i)
    params = torch.load('./shadow_model_ckpt/mnist/models%d/shadow_benign_0.model' % i)
    import model_lib.mnist_cnn_model as father_model
    Model = load_spec_model(father_model, str(i))
    print(i)
    m = Model()
    m.load_state_dict(params)
    mnist_models.append(m)
print("mnist model loaded, count: %d" % len(mnist_models))

# for i in range(6):
#     params = torch.load('/home/ubuntu/date/hdd4/shadow_model_ckpt/cifar10/models%d/shadow_benign_0.model'%i)
#     import model_lib.cifar10_cnn_model as father_model
#     Model = load_spec_model(father_model, str(i))
#     m = Model()
#     m.load_state_dict(params)
#     cifar10_models.append(m)
# print("cifar10 model loaded, count: %d" % len(cifar10_models))

/home/dorian/.conda/envs/mntd/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0
1
2
3
4
5
mnist model loaded, count: 6


In [23]:
import re
from torchinfo import summary
def model_summary(model_type, models, input_size):
    # get info:
    # 1. max node size
    # 2. conv layer node name & num & max_pooling params
    # 3. dense layer node name 
    # final save all info as a json file 
    max_size = None
    model_summary = {}
    cnt = 0
    row_dim = {}
    col_dim = {}
    for m in models:
        print("processing model: %d" % models.index(m))
        batch_size = (1,)
        summary_str = str(summary(m, batch_size+input_size))
        # print(summary_str)
        pattern = '├─(.*?)\['
        maxp_info = re.findall(pattern, summary_str)
        print(maxp_info)
        idx = 0
        layers = []
        pool1_used = False
        cur_row_dim = []
        cur_col_dim = []
        # print(m.state_dict().keys())
        for layer in m.state_dict().keys():
            
            cur_layer_name = layer.split('.')[0]
            # print("layer:", cur_layer_name)

            if 'bias' in layer:
                continue
            # if conv, max_pool exists?
            if 'conv' in layer:
                print("mpinfo next:", maxp_info[idx+1])
                if 'MaxPool' in maxp_info[idx+1]:
                    idx += 1
                    # print('maxpool conv layer')
                    mp_name = 'max_pool'
                    mp_info = {}
                    if pool1_used == False and model_type == 'mnist' and 'Model1' in summary_str:
                        mp_name = 'max_pool_1'
                        pool1_used = True
                    if pool1_used == True and model_type == 'mnist' and 'Model1' in summary_str:
                        mp_name = 'max_pool_2'
                    
                    mp_info['kernel_size'] = m.get_submodule(mp_name).kernel_size
                    mp_info['stride'] = m.get_submodule(mp_name).stride
                    mp_info['padding'] = m.get_submodule(mp_name).padding
                    mp_info['dilation'] = m.get_submodule(mp_name).dilation
                    mp_info['ceil_mode'] = m.get_submodule(mp_name).ceil_mode
                    layers.append({'name':cur_layer_name, 'num':m.get_submodule(cur_layer_name).out_channels,
                                  'maxpool':mp_info})
                else:
                    # print("no maxpool layer")
                    layers.append({'name':cur_layer_name, 'num':m.get_submodule(cur_layer_name).out_channels,
                                  'maxpool':None})
                cur_row_dim.append(m.get_submodule(cur_layer_name).kernel_size[0])
                cur_col_dim.append(m.get_submodule(cur_layer_name).kernel_size[1])
            else:
                # dense layer
                # print('dense layer')
                layers.append({'name':cur_layer_name, 'num': 1, 'maxpool':None})
                cur_row_dim.append(m.get_submodule(cur_layer_name).out_features)
                cur_col_dim.append(m.get_submodule(cur_layer_name).out_features)
            
            # get max row & col size
            # if cur_row_dim > row_dim:
            #     row_dim = cur_row_dim
            # if cur_col_dim > col_dim:
            #     col_dim = cur_col_dim
            idx += 1
        row_dim[cnt] = cur_row_dim
        col_dim[cnt] = cur_col_dim
        model_summary[cnt] =  layers
        cnt += 1
        # print(layers)
    return model_summary, row_dim, col_dim

mnist_summary, m_row_dim, m_col_dim = model_summary('mnist', mnist_models, (1, 28, 28))
def max_dim_count(mnist_summary, m_row_dim, m_col_dim):
    for model in range(len(m_row_dim)):
        max_dim = 0
        cell_num = 0
        for cell in range(len(m_row_dim[model])):
            # print(m_row_dim[model][cell], m_col_dim[model][cell])
            dim = m_row_dim[model][cell] * m_col_dim[model][cell]
            if max_dim < dim:
                max_dim = dim
            cell_num += mnist_summary[model][cell]['num']
        print("%d: num of cells: %d, max cell dim: %d" % (model, cell_num, max_dim))
# print(m_row_dim, m_col_dim)
# max_dim_count(mnist_summary, m_row_dim, m_col_dim)
# cifar10_summary, c_row_dim, c_col_dim = model_summary('cifar10', cifar10_models, (3, 32, 32))
# print(cifar10_summary, c_row_dim, c_col_dim)
# max_dim_count(cifar10_summary, c_row_dim, c_col_dim)

processing model: 0
['Conv2d: 1-1                            ', 'MaxPool2d: 1-2                         ', 'Conv2d: 1-3                            ', 'MaxPool2d: 1-4                         ', 'Linear: 1-5                            ', 'Linear: 1-6                            ']
mpinfo next: MaxPool2d: 1-2                         
mpinfo next: MaxPool2d: 1-4                         
processing model: 1
['Conv2d: 1-1                            ', 'MaxPool2d: 1-2                         ', 'Conv2d: 1-3                            ', 'MaxPool2d: 1-4                         ', 'Conv2d: 1-5                            ', 'MaxPool2d: 1-6                         ', 'Linear: 1-7                            ', 'Linear: 1-8                            ']
mpinfo next: MaxPool2d: 1-2                         
mpinfo next: MaxPool2d: 1-4                         
mpinfo next: MaxPool2d: 1-6                         
processing model: 2
['Conv2d: 1-1                            ', 'Conv2d: 1-2               

In [24]:
mnist_models[5]

Model5(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=392, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=10, bias=True)
)

In [27]:
import json
def get_model_detail():
    # get_model_detail information and save to json
    mnist_summary, m_row_dim, m_col_dim = model_summary('mnist', mnist_models, (1, 28, 28))
    cifar10_summary, c_row_dim, c_col_dim = model_summary('cifar10', cifar10_models, (3, 32, 32))
    max_row_dim = 512
    max_col_dim = 512
    model_detail = {}
    model_detail['mnist'] = mnist_summary
    model_detail['cifar10'] = cifar10_summary
    model_detail['max_size'] = [max_row_dim, max_col_dim]
    print(model_detail)
    model_detail_path = "./intermediate_data/model_detail.json"
    with open(model_detail_path, 'w') as f:
        json.dump(model_detail, f)
    return model_detail
model_detail = get_model_detail()

processing model: 0
['Conv2d: 1-1                            ', 'MaxPool2d: 1-2                         ', 'Conv2d: 1-3                            ', 'MaxPool2d: 1-4                         ', 'Linear: 1-5                            ', 'Linear: 1-6                            ']
mpinfo next: MaxPool2d: 1-2                         
mpinfo next: MaxPool2d: 1-4                         
processing model: 1
['Conv2d: 1-1                            ', 'MaxPool2d: 1-2                         ', 'Conv2d: 1-3                            ', 'MaxPool2d: 1-4                         ', 'Conv2d: 1-5                            ', 'MaxPool2d: 1-6                         ', 'Linear: 1-7                            ', 'Linear: 1-8                            ']
mpinfo next: MaxPool2d: 1-2                         
mpinfo next: MaxPool2d: 1-4                         
mpinfo next: MaxPool2d: 1-6                         
processing model: 2
['Conv2d: 1-1                            ', 'Conv2d: 1-2               

In [ ]:
import json
model_detail = []
model_detail_path = "./intermediate_data/model_detail.json"
with open(model_detail_path, 'r') as f:
    model_detail = json.load(f)

max_num = 0
num_list = []
for key, value in zip(model_detail.keys(), model_detail.values()):
    if key not in ['mnist', 'cifar10']:
        continue
    cur_num = 0
    print("Models %d" % cur_num)
    for model in value.values():
        for layer in model:
            print(layer)
            cur_num += layer['num']
        if cur_num > max_num:
            max_num = cur_num
            num_list.append(cur_num)
print(max_num)
print(num_list)

In [ ]:
for weight, bias in zip(m.get_submodule('conv1').weight, 
                        m.get_submodule('conv1').bias):
    print(weight[0])
    print(bias)
    w = weight[0] + bias
    print(w)
    break

In [ ]:
import torch
import torch.nn.functional as F

def padding(src: torch.Tensor, row: int, col: int):
    r, c = src.size()
    
    if r > row or c > col:
        return None
    
    if (row - r) % 2 == 0:
        rl = int((row - r) / 2)
    else:
        rl = int((row - r) / 2 + 1)
    
    if row == r:
        rr = 0
    else:
        rr = int((row - r) / 2)

    if (col - c) % 2 == 0:
        cl = int((col - c) / 2)
    else:
        cl = int((col - c) / 2 + 1)

    if col == c:
        cr = 0
    else:
        cr = int((col - c) / 2)
   
    result = F.pad(input=src, pad=(cl, cr, rl, rr), mode='constant', value=0)
    return result


In [ ]:
# cnn2graph
# mnsit input size: 1,28,28
# cifar10 input size: 3,32,32
from lzma import MODE_NORMAL
from torchinfo import summary
import os 
import dgl


def all_u_to_v(src, dst):
    ret = []
    for v in dst:
        ret += [[u, v] for u in src]
    return ret

def cnn2graph(model, model_info):
    # convert cnn model to a dgl graph
    # model: model weight data
    # model_info: model struct info
    
    layers = []
    all_node_feats = []
    all_edges = []
    cnt = 0
    with torch.no_grad():
        for i in range(len(model_info)):
            cur_layer_info = model_info[i]
            cur_layer_node = []
            
            if 'conv' in cur_layer_info['name']:
                # construct cur layer nodes
                for weight, bias in zip(model.get_submodule(cur_layer_info['name']).weight, 
                                        model.get_submodule(cur_layer_info['name']).bias):
                    cur_layer_node.append(cnt)
                    cnt += 1
                    # featue resize?
                    w = weight[0] + bias
                    print(weight[0].size(), bias)
                    all_node_feats.append(padding(w, 512, 513))
            else:
                # construct dense layer node
                cur_layer_node.append(cnt)
                cnt += 1
                # feature resize?
                w = model.get_submodule(cur_layer_info['name']).weight.t() + model.get_submodule(cur_layer_info['name']).bias
                print(cur_layer_info['name'], w.size())
                all_node_feats.append(padding(w, 512, 513))
            layers.append(cur_layer_node)
            
    # get all edges
    for idx in range(len(layers)):
        if idx < len(layers) - 1:
            edges = all_u_to_v(layers[idx], layers[idx+1])
            all_edges += edges
    
    all_edges = torch.tensor(all_edges).t()
    u, v = all_edges[0], all_edges[1]
    g = dgl.graph((u,v)).to('cuda')
    g.ndata['x'] = torch.stack(all_node_feats)
    return g
import json
model_detail = None
with open('./intermediate_data/model_detail.json') as f:
    model_detail = json.load(f)

print(model_detail)
import torch
from model_lib.mnist_cnn_model import Model0
for idx in range(len(mnist_models)):
    # if idx in [2, 3, 4]:
    #     continue
    # print("idx: %d" % idx)
    g = cnn2graph(mnist_models[0], model_detail['mnist'][str(idx)])
    break
    

In [ ]:
len(mnist_models)

In [ ]:
for idx in range(len(cifar10_models)):
    if idx in [0, 2, 3, 5]:
        continue
    print("idx: %d" % idx)
    g = cnn2graph(cifar10_models[idx], model_detail['cifar10'][str(idx)])


In [ ]:
zeros = torch.zeros((4,4))
ones = torch.ones((2, 2))
print(zeros)
print(ones)

In [ ]:
res = padding(ones, 4, 5)
print(res.size())